In [ ]:
from utils.utils import *
import voyageai
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import PyMuPDFLoader

load_dotenv(".env")

VOYAGE_API_KEY = os.environ.get("VOYAGE_API_KEY")



In [ ]:
loader = PyMuPDFLoader("data/scopic_pdf.pdf")

pdf_data = loader.load()

pdf_data

data = []

def clean_pdf(content):
    content = re.sub(r'\s+', ' ', content)
    lines = [line.strip() for line in content.splitlines() if line.strip()]
    cleaned_content = '\n'.join(lines)
    return cleaned_content

for page in pdf_data:
    _text = page.page_content
    text = clean_pdf(_text)
    print(text)

    data.append(text)

In [ ]:
data

In [ ]:


log_format = '%(asctime)s %(levelname)s: \n%(message)s\n'

logging.basicConfig(filename="/Users/juanreyesgarcia/Dev/Python/RAG/logging.log",
	level=logging.INFO,
	format=log_format)

def raw_to_batches(data: list, max_tokens: int = 512, print_messages: bool = True) -> list:
	batches = []
	total_tokens = 0
	truncation_counter = 0  # Counter for truncations

	for entry in data:
		#text = " ".join(i)  # Join the elements of the list into a single string
		tokens_description = num_tokens(entry)
		if tokens_description <= max_tokens:
			batches.append(entry)
		else:
			#TRUNCATE IF STRING MORE THAN x TOKENS
			job_truncated = truncated_string(entry, model="gpt-3.5-turbo", max_tokens=max_tokens)
			batches.append(job_truncated)
			truncation_counter += 1

		total_tokens += num_tokens(entry)

	approximate_cost = 0

	average_tokens_per_batch = total_tokens / len(batches)
	content = f"TOTAL NUMBER OF BATCHES: {len(batches)}\n" \
			f"TOTAL NUMBER OF TOKENS: {total_tokens}\n" \
			f"MAX TOKENS PER BATCH: {max_tokens}\n" \
			f"NUMBER OF TRUNCATIONS: {truncation_counter}\n" \
			f"AVERAGE NUMBER OF TOKENS PER BATCH: {average_tokens_per_batch}\n" \
			f"APPROXIMATE COST OF EMBEDDING: ${approximate_cost} USD\n"
	

	logging.info(f"\nRAW BATCHES SPECS: -------\n{content}")

	if print_messages:
		for i, batch in enumerate(batches, start=1):
			print(f"Batch {i}:")
			print("".join(batch))
			print(f"Tokens per batch:", num_tokens(batch))
			print("\n")

		print(content)
	
	return batches


batches = raw_to_batches(data)

In [ ]:


vo = voyageai.Client(api_key=VOYAGE_API_KEY)

result = vo.embed(batches, model="voyage-2", input_type="document", truncation=False)

pd_data = {
    "chunks": batches,
    "embeddings": result.embeddings
}

df = pd.DataFrame(pd_data)

print(df)

In [ ]:
to_postgre(df)